In this second part of the lab, we will implement a language identifier trained on the same data, but using Logistic Regression instead of Naive Bayes.

In [11]:
import io, sys, math
import numpy as np
from collections import defaultdict
from tqdm.notebook import tqdm

This function is used to build the dictionary, or vocabulary, which is a mapping from strings (or words) to integers (or indices). This will allow to build vector representations of documents. 

In [4]:
def build_dict(filename, threshold=1):
    fin = io.open(filename, 'r', encoding='utf-8')
    word_dict, label_dict = {}, {}
    counts = defaultdict(lambda: 0)
    for line in fin:
        tokens = line.split()
        label = tokens[0]

        if not label in label_dict:
            label_dict[label] = len(label_dict)

        for w in tokens[1:]:
            counts[w] += 1
            
    for k, v in counts.items():
        if v > threshold:
            word_dict[k] = len(word_dict)
    return word_dict, label_dict

This function is used to load the training dataset, and build vector representations of the training examples. In particular, a document or sentence is represented as a bag of words. Each example correspond to a sparse vector ` x` of dimension `V`, where `V` is the size of the vocabulary. The element `j` of the vector `x` is the number of times the word `j` appears in the document.

In [5]:
def load_data(filename, word_dict, label_dict):
    fin = io.open(filename, 'r', encoding='utf-8')
    data = []
    dim = len(word_dict)
    for line in fin:
        tokens = line.split()
        label = tokens[0]

        yi = label_dict[label]
        xi = np.zeros(dim)
        for word in tokens[1:]:
            if word in word_dict:
                wid = word_dict[word]
                xi[wid] += 1.0
        data.append((yi, xi))
    return data

First, let's implement the softmax function. Don't forget numerical stability!

In [6]:
def softmax(x):
    max_power = np.max(x)
    x-= max_power
    return np.exp(x)/sum(np.exp(x))

Now, let's implement the main training loop, by using stochastic gradient descent. The function will iterate over the examples of the training set. For each example, we will first compute the loss, before computing the gradient and performing the update.

In [7]:
def sgd(w, data, niter):
    
    lr = 0.001
    target = np.array([])
    nlabels, dim = w.shape
           
    for iter in range(niter):
        ### FILL CODE
        with tqdm(total = len(data)) as bar:        
          for y,x in data:
            y_onehot = np.zeros((10,1))
            y_onehot[y] =1
            y_hat = softmax(w@x)
            loss = -np.mean(y*np.log(y_hat) - (1-y)*np.log(1-y_hat))
            bar.set_description(f"Epoch {iter+1}/{niter} Loss {loss}")
            bar.update()
            gradient = x.reshape(-1,1)@(y_hat-y).reshape(-1,1).T/len(data)
            w = w - lr * gradient.T
    return w

The next function will predict the most probable label corresponding to example `x`, given the trained classifier `w`.

In [19]:
def predict(w, x):
    ## FILL CODE
    y= softmax(w@x)
    return np.argmax(y)

Finally, this function will compute the accuracy of a trained classifier `w` on a validation set.

In [9]:
def compute_accuracy(w, valid_data):
    ## FILL CODE
    acc = 0
    for y,x in valid_data:
      y_hat = predict(w,x)
      acc = acc+1 if y == y_hat else acc
    return acc/len(valid_data)  


In [ ]:
print("")
print("** Logistic Regression **")
print("")

word_dict, label_dict = build_dict("train1.txt")
train_data = load_data("train1.txt", word_dict, label_dict)
valid_data = load_data("valid1.txt", word_dict, label_dict)

nlabels = len(label_dict)

dim = len(word_dict)
w = np.zeros([nlabels, dim])
w = sgd(w, train_data, 5)
print("")
print("Validation accuracy: %.3f" % compute_accuracy(w, valid_data))
print("")


** Logistic Regression **



  0%|          | 0/10000 [00:00<?, ?it/s]